# *** Choose your configuration: 0 docker, 1 cloudveneto ***

In [ ]:
#0 Docker, 1 CloudVeneto
FIGHTER = 1

In [ ]:
# la porta dove vedere i jobs --- METTERE LA PROPRIA QUI ---
SparkUI = 4043

# nicolò 4040
# marco  4041
# francesco 4042
# raffaele  4043

# MASTER 8080 E' PER TUTTI

<span style="font-size: 150%">Porta 8080, voce Running Applications: puoi vedere se qualcuno ci sta lavorando.
<br>
Porta 8080, voce Workers: vedresti core e memoria usata se c'è qualcun'altro.</span>

In [ ]:
user = "user_b" # "user_a"  o  "user_b" (B SE SI E' I SECONDI A LAVORARCI)

In [ ]:
core = 8 # 8 di 16, lasciare così se si lavora in due

In [ ]:
Npartition = 8

# *** Remember to close Spark Session ***

In [ ]:
spark.stop()

sudo pkill -u $(whoami) -f "jupyter-notebook"

<hr style="height:4px; background-color:black; border:none;" />

# Creation of the Spark Session and Context

In [ ]:
# Standard libraries
import os
import numpy as np
import pandas as pd
from math import ceil
import matplotlib.pyplot as plt

# PySpark core
from pyspark import SparkFiles
from pyspark.sql import SparkSession, DataFrame, Window
from pyspark.sql.types import IntegerType

# PySpark functions
from pyspark.sql.functions import (
    col, lit, expr, when, count, sum as spark_sum, abs as abs_,
    round as spark_round, min as spark_min, max as spark_max, avg as spark_avg,
    first, last, lag, row_number, desc, asc,
    explode, sequence, from_unixtime, to_date, unix_timestamp,
    window, min_by, mode, concat, monotonically_increasing_id
)

In [ ]:
if FIGHTER==0:

    spark = SparkSession.builder \
        .master("spark://spark-master:7077") \
        .appName("ProjectDocker") \
        .config("spark.executor.memory", "1000m") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
        .getOrCreate()

        # SE NON FUNZIONA TOGLI I DUE CONFIG DI ARROW

        # .config("spark.executor.memory", "1500m")
        # .config("spark.executor.cores", "1")\
        # .config("spark.executor.instances", "12")\
        # .config("spark.cores.max", "12")\
        # .config("spark.default.parallelism", "24")\
        # .config("spark.sql.adaptive.enabled", "true")\

elif FIGHTER==1:

        os.environ["PYSPARK_PYTHON"] = "/opt/miniconda3/bin/python"
        os.environ["PYSPARK_DRIVER_PYTHON"] = "/opt/miniconda3/bin/python"
        
        spark = SparkSession.builder \
            .appName("ProjectCloudVeneto") \
            .master("spark://10.67.22.135:7077") \
            .config("spark.scheduler.mode", "FAIR") \
            .config("spark.scheduler.pool", user) \
            .config("spark.scheduler.allocation.file", "file:///usr/local/spark/conf/fairscheduler.xml") \
            .config("spark.cores.max", core) \
            .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
            .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
            .config("spark.dynamicAllocation.enabled", "false") \
            .config("spark.shuffle.service.enabled", "false") \
            .config("spark.ui.port", SparkUI) \
            .getOrCreate()
    
else : print("Better choose an available fighter, you little bastard.")

sc = spark.sparkContext

# Dataset upload and partition

### General Dataset

In [ ]:
if FIGHTER==0:
    df = spark.read.option("header", True).option("inferSchema", True).csv("/ProvePreliminari/SW-106.csv")

elif FIGHTER==1:
    df = spark.read.option("header", True).option("inferSchema", True).csv("file:///mnt/shared/dataset.csv")

else:
    print("Better choose an available fighter, you little bastard")

df = df.repartition(Npartition)

In [ ]:
# inspect dataset
df.printSchema()

In [ ]:
# inspect dataset
df.show(10, truncate=False)

### Focus on one hardware at time

In [ ]:
# convert milliseconds into seconds
df = df.withColumn("when", spark_round(col("when") / 1000).cast(IntegerType()))

In [ ]:
hwid_list = df.select("hwid").distinct()
hwid_list.show()

In [ ]:
# CHANGE HERE HARDWARE TO ANALYZE
hardware = "SW-106"
df_hw = df  .filter(col("hwid") == hardware)\
            .groupBy("when")\
            .pivot("metric")\
            .agg(first("value"))\
            .withColumn("time", from_unixtime(col("when")))\
            .orderBy("when")

In [ ]:
# momentarily persist this dataframe, then we'll unpersist
df_hw = df_hw.persist()

In [ ]:
# we trigger persist transformation
df_hw.show(1, truncate=False)

In [ ]:
df_hw.printSchema()

In [ ]:
selected_cols = ["when", "time", "S117", "S118", "S169", "S170"]

In [ ]:
# show first 5 rows
df_hw.select(*selected_cols).orderBy(col("time").asc()).show(5)

In [ ]:
# show last 5 rows
df_hw.select(*selected_cols).orderBy(col("time").desc()).limit(5).orderBy(col("time").asc()).show()

Usefull function to inspect a general dataframe

In [ ]:
def inspect(df: DataFrame, sensors: list, start: int, end: int) -> DataFrame:
    """
    Filter a DataFrame by time range and select specified sensor columns.

    Args:
        df:        Input Spark DataFrame with 'when' and 'time' columns.
        sensors:   List of sensor column names to include (e.g., ['S117', 'S118']).
        start:  Start of the time range (Unix timestamp, in seconds).
        end:    End of the time range (Unix timestamp, in seconds).

    Returns:
        Filtered Spark DataFrame with columns ['time', 'when', ...sensors].
    """
    selected_columns = ["when","time"] + sensors

    return (
        df.select(*selected_columns)
          .filter((col("when") >= start) & (col("when") <= end))
    )

In [ ]:
inspect(df_hw, sensors=["S117"], start=1601526622, end=1601531000).show(10)

# Timestamp analysis

In [ ]:
def compute_time_differences(df, when_col="when", max_collect=1_000_000):
    """
    Computes time differences (Δwhen) between consecutive rows in a DataFrame.

    Args:
        df (DataFrame): Spark DataFrame with a time column (e.g., 'when').
        when_col (str): Name of the time column to compute differences on.
        max_collect (int): Threshold to safely collect diffs to driver.

    Returns:
        rdd_diff (RDD): RDD of differences (current - previous).
    """
    rdd_times = df.select(when_col).rdd.map(lambda row: row[when_col])
    rdd_shifted = rdd_times.zipWithIndex().map(lambda x: (x[1], x[0]))  # (index, time)
    rdd_prev = rdd_shifted.map(lambda x: (x[0] + 1, x[1]))              # shift by +1 index
    rdd_joined = rdd_shifted.join(rdd_prev).sortByKey()
    rdd_diff = rdd_joined.map(lambda x: x[1][0] - x[1][1])

    num_diffs = rdd_diff.count()
    print(f"Number of time differences: {num_diffs}")

    if num_diffs < max_collect:
        return rdd_diff
    else:
        print("Too many differences to collect safely.")
        return None

In [ ]:
def time_diff_summary(rdd_diff, spark, top_n=5):
    """
    Summarizes the RDD of time differences into a frequency table and prints top/bottom values.

    Args:
        rdd_diff (RDD): RDD of integer time differences.
        spark (SparkSession): Active Spark session.
        top_n (int): Number of rows to show from top and bottom.

    Returns:
        df_freq (DataFrame): DataFrame with columns ['diff', 'count'].
    """
    if rdd_diff is None:
        print("No differences available to summarize.")
        return None

    df_freq = rdd_diff.map(lambda d: (d, 1)) \
                      .reduceByKey(lambda a, b: a + b) \
                      .toDF(["diff", "count"]) \
                      .orderBy("diff")

    print(f"\n{top_n} smallest time differences:")
    df_freq.show(top_n, truncate=False)

    print(f"\n{top_n} largest time differences:")
    df_freq.orderBy("diff", ascending=False).show(top_n, truncate=False)

    return df_freq

In [ ]:
# Compute time differences
rdd_diff = compute_time_differences(df_hw)

# Summarize and print top/bottom time gaps
df_diff_summary = time_diff_summary(rdd_diff, spark)

In [ ]:
def plot_diff_histogram(differences, min_diff, max_diff, delta):
    
    filtered = [d for d in differences if min_diff <= d <= max_diff]

    bins = int((max_diff - min_diff) / delta)

    plt.figure(figsize=(10, 6))
    plt.hist(filtered, bins=bins, color="steelblue", edgecolor="black")
    plt.title(f"Time Difference Events (range {min_diff} - {max_diff} [s])")
    plt.xlabel("Time difference [s]")
    plt.ylabel("Counts")
    #plt.grid(True)
    plt.show()

In [ ]:
diffs = rdd_diff.collect() # check previously the dimension

In [ ]:
plot_diff_histogram(differences=diffs, min_diff=0, max_diff=180, delta=2)

In [ ]:
plot_diff_histogram(differences=diffs, min_diff=180, max_diff=4000, delta=25)

# Handling missing data & gaps

### Create final grid

Direi di tenere questa funzione "FillGaps" e continuare ad implementare partendo da questa

In [ ]:
frequency = 60 #seconds

In [ ]:
def FillGaps(
    df: DataFrame,
    sensors: list = None,
    interval: int = 60,
    modality: str = 'auto',
    fill_null: bool = False
) -> DataFrame:
    """
    Aggregates sensor data into fixed-size time windows and fills missing values using Spark.

    Args:
        df: Spark DataFrame with a 'when' column (UNIX timestamp in seconds).
        sensors: List of sensor column names. If None, inferred from all columns except 'when' and 'time'.
        interval: Window size in seconds.
        modality: Aggregation method: 'mean', 'min', 'max', 'mode', or 'auto'.
        fill_null: If True, fills missing values using forward and backward fill in Spark.

    Returns:
        Aggregated and optionally gap-filled DataFrame, with a `when` column at the center of the window
        and a `window_id` column that uniquely identifies each time window.
    """

    # 1. Infer sensor columns if not provided
    if sensors is None:
        sensors = [c for c in df.columns if c not in ("when", "time")]

    # 2. Add timestamp column
    df_ts = df.withColumn("timestamp", from_unixtime(col("when")).cast("timestamp"))

    # 3. Create time window
    df_windowed = df_ts.withColumn("time_window", window("timestamp", f"{interval} seconds"))

    # 4. Aggregate using selected modality
    if modality == "mode":
        # Special case: MODE needs groupBy and count per window + sensor
        aggs = []
        for s in sensors:
            mode_df = (
                df_windowed.groupBy("time_window", s)
                .agg(count("*").alias("cnt"))
                .withColumn("rank", row_number().over(
                    Window.partitionBy("time_window").orderBy(desc("cnt"))
                ))
                .filter(col("rank") == 1)
                .select("time_window", col(s).alias(s))
            )
            if not aggs:
                result_df = mode_df
            else:
                result_df = result_df.join(mode_df, on="time_window", how="outer")
    else:
        aggs = []
        for s in sensors:
            if modality == "mean":
                agg_func = spark_avg(col(s)).alias(s)
            elif modality == "min":
                agg_func = spark_min(col(s)).alias(s)
            elif modality == "max":
                agg_func = spark_max(col(s)).alias(s)
            elif modality == "auto":
                stats = df.selectExpr(f"min({s}) as min", f"max({s}) as max").first()
                is_binary = stats["min"] is not None and stats["max"] is not None and 0 <= stats["min"] and stats["max"] <= 1
                if s in ["A5", "A9"] or is_binary:
                    agg_func = spark_max(col(s)).alias(s)
                else:
                    agg_func = spark_avg(col(s)).alias(s)
            else:
                raise ValueError(f"Unsupported modality: {modality}")
            aggs.append(agg_func)

        result_df = (
            df_windowed
            .groupBy("time_window")
            .agg(*aggs)
        )

    # 5. Add window_start, window_end, and 'when' as center of window
    result_df = (
        result_df
        .withColumn("window_start", col("time_window.start"))
        .withColumn("window_end", col("time_window.end"))
        .withColumn("when", expr("unix_timestamp(window_start) + int((unix_timestamp(window_end) - unix_timestamp(window_start)) / 2)"))
        .drop("time_window")
        .orderBy("when")
    )

    # 6. Fill nulls if requested using Spark-native ffill + bfill
    if fill_null:
        w_forward = Window.orderBy("when").rowsBetween(Window.unboundedPreceding, 0)
        w_backward = Window.orderBy("when").rowsBetween(0, Window.unboundedFollowing)
        for s in sensors:
            result_df = result_df.withColumn(s, last(col(s), ignorenulls=True).over(w_forward))
            result_df = result_df.withColumn(s, first(col(s), ignorenulls=True).over(w_backward))

    # 7. Add window_id as progressive row number
    result_df = result_df.withColumn("window_id", monotonically_increasing_id())

    return result_df.select(["window_id", "when", "window_start", "window_end"] + sensors)

    # 6. Optionally fill nulls using Pandas (costly)
    # if fill_null:
    #     pandas_df = result_df.toPandas()
    #     pandas_df = pandas_df.ffill().bfill()
    #     return spark.createDataFrame(pandas_df)
    # else:
    #     return result_df

Already added conversion of A5 and A9 sensors, and overheating control function.

In [ ]:
def extract_alarm_bits(df: DataFrame, columns=["A5", "A9"], bits=[6, 7, 8]) -> DataFrame:
    """
    Extracts specific bits (1-indexed, left to right) from the given integer alarm columns
    and adds them as new columns in the DataFrame.

    Args:
        df: Input Spark DataFrame.
        columns: List of alarm column names (e.g., ["A5", "A9"]).
        bits: List of bit positions to extract (left-to-right, 1-based index).

    Returns:
        DataFrame with additional columns like "6-A5", "7-A9", etc.
    """
    for col_name in columns:
        for bit in bits:
            # Convert left-to-right to right-to-left (bit 1 is MSB → position 15)
            bit_from_right = 16 - bit
            df = df.withColumn(
                f"{bit}-{col_name}",
                ((col(col_name).bitwiseAND(1 << bit_from_right)) > 0).cast("int")
            )
    return df

In [ ]:
def add_overheating_flag(df: DataFrame) -> DataFrame:
    """
    Adds a binary column 'overheating' which is 1 if any of bits 6, 7, or 8 of A5 or A9 is 1.

    Args:
        df: Spark DataFrame with bit columns already extracted.

    Returns:
        DataFrame with 'overheating' column added.
    """
    overheating_bits = [f"{b}-{s}" for s in ["A5", "A9"] for b in [6, 7, 8]]
    condition = sum([col(c) for c in overheating_bits]) > 0
    return df.withColumn("overheating", when(condition, 1).otherwise(0))

In [ ]:
# %%time

# #2° MODALITA' USO FUNZIONE FillGaps
# #Doesn't seem to be particularly dependent on the number of metrics (9 seconds with 1, 13 seconds with 15)
# sensors=['S117', 'S118',  'S169', 'S170', 'S41', 'ComError']#, 'A5', 'A9', 'P18', 'P2', 'P5', 'P6', 'P7', 'P8', 'P9', 'S1']
# GridDF = FillGaps(df_hw, sensors=sensors, modality="min", interval=60)
# GridDF.show(10, truncate = False)

In [ ]:
%%time

#1° MODALITA' USO FUNZIONE FillGaps
# Fill sensor gaps
df_grid = FillGaps(df_hw, interval=frequency, modality="auto", fill_null=False)

# Extract bits from alarms
df_grid = extract_alarm_bits(df_grid, columns=["A5", "A9"], bits=[6, 7, 8])

# Add 'overheating' column
# --- IMPLEMENTATA GIA (E' RICHIESTA PIU' AVANTI) ORA NON SERVE
# df_final = add_overheating_flag(df_final)

# Now persist the result
df_final = df_grid.persist()

# Trigger persist
df_final.show(1, truncate = False)

# Unpersist original dataframe
# --- PER IL MOMENTO DIREI DI TENERLO CON IL PERSIST CHE E' PRATICO PER LA FASE DI SVILUPPO PER RICONTROLLARE LE COSE SUL DATAFRAME ORIGINARIO
# df_hw.unpersist()

In [ ]:
df_final.select("window_id","when","window_start","window_end","S110","S112","S113","S114","S115","S117","S118","S122","A9","8-A9").show(30, truncate = False)

In [ ]:
inspect(df_hw, sensors=["S110","S112","S113","S114","S115","S117","S118","S122","A9"], start=1601510400, end=1601512550).show(200)

### Block distribution

In [ ]:
def build_blocks(df_final: DataFrame, frequency: int) -> DataFrame:
    """
    Build blocks using existing window_start for partitioning.
    
    Args:
        df_final: Input Spark DataFrame with 'when' and 'window_start' columns.
        frequency: Maximum allowed time gap in seconds.
    Returns:
        DataFrame with columns: 'window_id', 'block_id'.
    """
    # 1) Extract date from existing window_start column for partitioning
    df_with_day = df_final.withColumn("day", to_date(col("window_start")))
    
    # 2) Window partitioned by day, ordered by when
    window_spec = Window.partitionBy("day").orderBy("when")
    
    # 3) Compute gap to previous timestamp within each day
    df_with_gap = (df_with_day
        .withColumn("prev_when", lag("when", 1).over(window_spec))
        .withColumn("gap", col("when") - col("prev_when"))
    )
    
    # 4) Flag start of new block
    df_with_flag = df_with_gap.withColumn(
        "is_new_block",
        when((col("prev_when").isNull()) | (col("gap") > frequency), 1).otherwise(0)
    )
    
    # 5) Create daily block_id with partitioned window
    df_with_daily_blocks = df_with_flag.withColumn(
        "daily_block_id",
        spark_sum("is_new_block").over(window_spec)
    )
    
    # 6) Create global block_id - get max blocks per day
    daily_max = (df_with_daily_blocks
        .groupBy("day")
        .agg(spark_max("daily_block_id").alias("max_block"))
    )
    
    # 7) Calculate day offsets with partitioned window
    day_window = Window.orderBy("day")
    daily_offsets = (daily_max
        .withColumn("prev_max", lag("max_block", 1).over(day_window))
        .withColumn("day_offset", 
            spark_sum(when(col("prev_max").isNull(), 0).otherwise(col("prev_max"))).over(day_window))
        .fillna(0, subset=["day_offset"])
        .select("day", "day_offset")
    )
    
    # 8) Join and create final block_id
    result = (df_with_daily_blocks
        .join(daily_offsets, "day")
        .withColumn("block_id", col("daily_block_id") + col("day_offset") - 1)
        .select("window_id", "block_id")
        .orderBy("when")  # Final sort by time
    )
    
    return result

In [ ]:
df_blocks = build_blocks(df_final, frequency=60)

In [ ]:
df_blocks.show(30)

In [ ]:
df_final.join(df_blocks, on="window_id", how="left").select("block_id","window_id","window_start").orderBy("window_id").show(30)

In [ ]:
df_block_duration = (
    df_blocks.groupBy("block_id")
    .count()
    .withColumn("duration_minutes", (col("count") * lit(frequency)) / 60)
    .drop("count")
    .orderBy("block_id")
)

In [ ]:
df_block_duration.show(30,truncate=False)

In [ ]:
df_block_stats = df_block_duration.groupBy(col("duration_minutes").alias("block_length")) \
    .agg(spark_count("*").alias("block_count")).orderBy("block_length")
df_block_stats.show()

In [ ]:
def plot_diff_histogram(df_block_stats, min_length=1, max_length=100, step=1):
    """
    Plot a histogram of contiguous block lengths.

    Args:
      df_block_stats: Spark DataFrame with ['block_length','block_count']
      min_length:     smallest block_length to include
      max_length:     largest block_length to include
      step:           bar width
    """
    pdf = (
        df_block_stats
        .filter((col("block_length") >= min_length) &
                (col("block_length") <= max_length))
        .orderBy("block_length")
        .toPandas()
    )

    if pdf.empty:
        print("No blocks in the specified range.")
        return

    plt.figure(figsize=(10,5))
    plt.bar(pdf["block_length"], pdf["block_count"], width=step, align="center")
    plt.xlabel("Block Length")
    plt.ylabel("Number of Blocks")
    plt.title("Histogram of Contiguous Block Lengths")
    plt.grid(True)
    plt.xlim(min_length, max_length)
    plt.show()

In [ ]:
plot_diff_histogram(df_block_stats, min_length=1, max_length=100, step=2)

### Anomaly detection (Raffaele)

In [ ]:
'''
def resample_sensors(df, sensors, time_int):
    
    # Crea dataframe con sensori ricampionati. 
    
    # Trovo istante iniziale
    min_time = df.agg(spark_min(unix_timestamp('time')).alias('min_time')).collect()[0]['min_time']

    # Creo lista di aggregatori da usare per mediare i dati vicini di ogni colonna
    aggs = [mode(col(s)).alias(f"mode_{s}") for s in sensors] # mode prende il valore più frequente nel gruppo

    # Creo df dove ogni dato della stessa colonna in [t, t+time_int] è mediato
    resampled_df = df \
        .select('time', *sensors) \
        .withColumn('window_idx', spark_round((unix_timestamp(col('time'))-min_time)/time_int)) \
        .groupBy('window_idx').agg(*aggs) \
        .withColumn('sampling_time', from_unixtime(col('window_idx')*time_int + min_time)) \
        .withColumn('when', unix_timestamp(col('sampling_time'))) \
        .orderBy('sampling_time')
    return resampled_df
'''

############################################################################


def detect_switch_anomalies(df, sensors, window_minutes=10, switch_threshold=5):
    """
    Crea un dataframe che contiene una flag per le anomalie di ogni sensore ogni window_minutes 
    NB: siccome la grid è spaziata di 1 minuto, window_minutes deve essere > 1 per avere senso
    """

    # Extract day (or hour) from timestamp to partition
    df = df \
        .select('when', *sensors, 'window_start', 'window_end') \
        .withColumn("day", to_date(col('window_start')))

    # Lag to get previous value within each partition --- OTHERWISE IT MOVES ALL IN THE SAME PARTITION 
    w_lag = Window.partitionBy("day").orderBy("when")
    lagged_columns = [lag(col(s)).over(w_lag) for s in sensors] # per ogni sensore prendo le righe shiftate di 1 all'indietro
    lag_names      = [f"lagged_{s}" for s in sensors]
    
    df_lagged = df.withColumns(dict(zip(lag_names, lagged_columns)))

    # Detect change 0→1 or 1→0
    didSwitch   = [when((col(f"lagged_{s}") != col(s)), 1).otherwise(0) for s in sensors] # per ogni sensore verifico se il valore è cambiato rispetto al precedente 
    switch_names = [f"switch_{s}" for s in sensors]
    
    df_changes = df_lagged.withColumns(dict(zip(switch_names, didSwitch)))

    # Rolling window over time with partition
    # Raffaele: così rischiamo di contare più volte la stessa anomalia.
     '''
    w_time = Window.partitionBy("day").orderBy("when").rangeBetween(-window_minutes * 60, 0)
    change_counts = [spark_sum(f"switch_{s}").over(w_time) for s in sensors]
    counts_names  = [f"change_count_{s}" for s in sensors]
    
    df_windowed = df_changes.withColumns(dict(zip(counts_names, change_counts)))
     '''
    w_group = Window.orderBy('when')
    w_count = Window.partitionBy('groupId')
    
    df_switch_group = (
        df_changes \
        .withColumn('switch_group', when(lag(col('switch_S117')) != col('switch_S117'), 1).otherwise(0)) \
        .withColumn('groupId', spark_sum('switch_group').over(w_group)) \
        .withColumn('')
    )

    # Flag anomaly
    anomalies       = [col(f"change_count_{s}") > switch_threshold for s in sensors]
    anomalies_names = [f"{s}_anomaly" for s in sensors]
    
    df_anomaly = df_windowed \
    .withColumns(dict(zip(anomalies_names, anomalies))) \
    .drop(*lag_names, *switch_names, *counts_names, "day", "window_idx")

    return df_anomaly

In [ ]:
sensors = ['S117', 'S118', 'S169', 'S170']
anomalies = detect_switch_anomalies(
  df= df_final,
  sensors=sensors,
  window_minutes=60,
  switch_threshold=6
)

# Per confronto vediamo se le anomalie del S117 corrispondono a quelle del metodo di Marco:
anomalies \
    .show(10, truncate=False)

# Questo non va ancora
#filters = [col(f"mode_{s}_anomaly") for s in sensors] 
#total_filter = filters[0]
#for i in range(1, len(filters)):
#    total_filter = total_filter & filters[i]


In [ ]:
def plot_anomaly_hist(df, sensors, time_start, time_end):
    '''
    Plotta il numero di anomalie al giorno per ogni sensore, nell'arco di tempo [time_start, time_end]
    '''

    casts = [col(f'{s}_anomaly').cast('Int') for s in sensors] # per convertire i booleani in S_anomaly in 0/1 
    aggs = [spark_sum(col(f'{s}_anomaly')).alias(f'anomaly_count_{s}') for s in sensors] # per contare le anomalie ogni giorno

    new_df = df \
    .select('when', 'window_start', 'window_end', *[f"{s}_anomaly" for s in sensors]) \
    .filter(col('when').between(time_start, time_end)) \
    .withColumn('day', to_date(col('window_start'))) \
    .withColumns(dict(zip([f'{s}_anomaly' for s in sensors], casts))) \
    .groupBy('day').agg(*aggs) \
    .orderBy('day')

    n_sensors = len(sensors)
    fig, axes = plt.subplots(ceil(n_sensors/2), 2, figsize=(20, 5))
    days = new_df.select('day').rdd.flatMap(lambda x: x).collect()
    for i in range(n_sensors):
        counts = new_df.select(f'anomaly_count_{sensors[i]}').rdd.flatMap(lambda x: x).collect()
        idx = i if n_sensors <= 2 else (i//2, i%2)
        axes[idx].bar(days, counts)
        axes[idx].set_title(f'anomaly_count_{sensors[i]}')
        axes[idx].set(xlabel="day", ylabel="counts")
        #print(sensors[i], counts)
    fig.tight_layout()
  

In [ ]:
plot_anomaly_hist(anomalies, ['S117', 'S118', 'S169', 'S170'], 1602879000, 1602879000+10000000)  

# <hr style="height:4px; background-color:black; border:none;" />

### Su solo una colonna (metodo nearest) FUNZIONANTE

In [ ]:
def resample_sensor_simple(
    df_hard: DataFrame,
    sensor: str,
    interval: int = 60
) -> DataFrame:
    """
    Resample a single sensor column to regular intervals using the nearest non-null value.

    Args:
        df_hard:   Spark DataFrame with columns ['when', sensor].
        sensor:    Name of the sensor column (e.g., 'S117').
        interval:  Resample interval in seconds.

    Returns:
        Resampled DataFrame with ['when', sensor, 'time'], using only non-null nearest values.
    """
    spark = df_hard.sparkSession
    half = interval / 2

    # 1. Create target timestamp grid
    bounds = df_hard.selectExpr("min(when) as min_t", "max(when) as max_t").first()
    min_t, max_t = bounds.min_t, bounds.max_t

    grid_df = (
        spark.range(1)
             .selectExpr(f"sequence({min_t}, {max_t}, {interval}) as times")
             .select(explode("times").alias("target_when"))
    )

    # 2. Range join within ±half interval and exclude NULLs before aggregation
    joined = (
        df_hard
          .select("when", sensor)
          .filter(col(sensor).isNotNull())  # <-- filter out nulls before min_by
          .join(
              grid_df,
              (col("when") >= col("target_when") - half) &
              (col("when") <= col("target_when") + half),
              how="inner"
          )
          .withColumn("time_diff", abs_(col("when") - col("target_when")))
    )

    # 3. Pick value with minimum time difference (nearest non-null)
    result = (
        joined.groupBy("target_when")
              .agg(min_by(col(sensor), col("time_diff")).alias(sensor))
    )

    # 4. Format output
    final = (
        result
          .withColumnRenamed("target_when", "when")
          .withColumn("time", from_unixtime(col("when")))
          .orderBy("when")
    )


    return final


In [ ]:
%%time

df_resampled_S117 = resample_sensor_simple(df_hard, "A5", interval=60)
df_resampled_S117.show(10, truncate=False)

### Various checks to show that everything works (questi li terrei)

In [ ]:
# inspect a specific region
start_ts = 1601526622
end_ts   = 1601531000

df_resampled_S117.filter(
    (col("when") >= start_ts) & 
    (col("when") <= end_ts)
).show()

In [ ]:
inspect(df_hard, sensors=["S117"], start=1601526622, end=1601531000).show()

In [ ]:
inspect(df_hard, sensors=["S117"], start=1601527100, end=1601529400).show()

In [ ]:
# Check everything is fine

# Compute time differences
rdd_diff_S117 = compute_time_differences(df_resampled_S117)

# Summarize and print top/bottom time gaps
df_diff_summary_S117 = time_diff_summary(rdd_diff_S117, spark)

In [ ]:
# questo dai dati di prima, sembra funzionare
# |5003774|1    |
# |2066908|1    |
# |757864 |1    |
# |48451  |1    |
# |28598  |1    |

### data blocks

In [ ]:
def count_contiguous_blocks(
    df: DataFrame,
    interval: int,
    col_when: str = "when"
) -> DataFrame:
    """
    Count contiguous blocks of rows spaced exactly `interval` seconds apart,
    partitioned by day for parallelism.

    Args:
      df:         Spark DataFrame with a timestamp column in seconds.
      interval:   Expected spacing between consecutive rows (in seconds).
      col_when:   Name of that timestamp column (default 'when').

    Returns:
      DataFrame with columns:
        - block_length: number of rows in each contiguous block
        - block_count:  how many such blocks exist
    """
    # 1) Add a 'day' column for partitioning
    df2 = df.withColumn("day", to_date(from_unixtime(col(col_when))))

    # 2) Window over each day, ordered by timestamp
    w = Window.partitionBy("day").orderBy(col(col_when))

    # 3) Compute the gap to the previous timestamp
    df3 = (
        df2
        .withColumn("prev_when", lag(col(col_when), 1).over(w))
        .withColumn("gap", col(col_when) - col("prev_when"))
    )

    # 4) Flag start of new block when gap != interval (or first in day)
    df4 = df3.withColumn(
        "is_new_block",
        expr(f"CASE WHEN gap IS NULL OR gap != {interval} THEN 1 ELSE 0 END")
    )

    # 5) Cumulative sum over the window to assign a block_id
    df5 = df4.withColumn(
        "block_id",
        sum_(col("is_new_block")).over(w)
    )

    # 6) Count rows per (day, block_id) → this is block_length
    blocks = (
        df5
        .groupBy("day", "block_id")
        .agg(count("*").alias("block_length"))
    )

    # 7) Count how many blocks have each length across all days
    result = (
        blocks
        .groupBy("block_length")
        .agg(count("*").alias("block_count"))
        .orderBy("block_length")
    )

    return result

In [ ]:
def plot_diff_histogram(df_block_stats, min_length=1, max_length=100, step=1):
    """
    Plot a histogram of contiguous block lengths.

    Args:
      df_block_stats: Spark DataFrame with ['block_length','block_count']
      min_length:     smallest block_length to include
      max_length:     largest block_length to include
      step:           bar width
    """
    pdf = (
        df_block_stats
        .filter((col("block_length") >= min_length) &
                (col("block_length") <= max_length))
        .orderBy("block_length")
        .toPandas()
    )

    if pdf.empty:
        print("No blocks in the specified range.")
        return

    plt.figure(figsize=(10,5))
    plt.bar(pdf["block_length"], pdf["block_count"], width=step, align="center")
    plt.xlabel("Block Length")
    plt.ylabel("Number of Blocks")
    plt.title("Histogram of Contiguous Block Lengths")
    plt.grid(True)
    plt.xlim(min_length, max_length)
    plt.show()

In [ ]:
# 1) Select only the timestamp column
df_sample = df_resampled_S117.select("when")

# 2) Compute contiguous block stats at 60s spacing
block_stats = count_contiguous_blocks(df_sample, interval=60)

# 3) Inspect
block_stats.show()

In [ ]:
# 4) Plot histogram for blocks length 1–1000
plot_diff_histogram(block_stats, min_length=1, max_length=100, step=2)

### Con vari metodi riempimento (near,min,max,mean)

In [ ]:
def resample_sensor(
    df_hard: DataFrame,
    sensor: str,
    interval: int = 60,
    method: str = "near"
) -> DataFrame:
    """
    Resample a single sensor column to regular intervals.

    Args:
      df_hard:   DataFrame with 'when' (in seconds) and the sensor column.
      sensor:    name of the sensor column (e.g. 'S117').
      interval:  interval in seconds (e.g. 60).
      method:    one of 'near', 'min', 'max', 'mean'.

    Returns:
      DataFrame with columns ['when', sensor, 'time'] at regular timestamps.
      All values are non-null. If no data is found for a window, the row is skipped.
    """
    spark = df_hard.sparkSession
    half = interval / 2

    # Validate method
    if method not in ("near", "min", "max", "mean"):
        raise ValueError(f"Unsupported method '{method}'. Choose from 'near', 'min', 'max', 'mean'.")

    # 1. Create time grid
    bounds = df_hard.selectExpr("min(when) as min_t", "max(when) as max_t").first()
    min_t, max_t = bounds.min_t, bounds.max_t

    grid_df = (
        spark.range(1)
             .selectExpr(f"sequence({min_t}, {max_t}, {interval}) as times")
             .select(explode("times").alias("target_when"))
    )

    # 2. Join with range ± half interval and exclude NULLs before aggregation
    filtered = df_hard.select("when", sensor).filter(col(sensor).isNotNull())

    joined = (
        filtered
          .join(
              grid_df,
              (col("when") >= col("target_when") - half) &
              (col("when") <= col("target_when") + half),
              how="inner"
          )
          .withColumn("time_diff", abs_(col("when") - col("target_when")))
    )

    # 3. Aggregate based on method
    if method == "near":
        agg_expr = min_by(col(sensor), col("time_diff")).alias(sensor)
    elif method == "min":
        agg_expr = spark_min(col(sensor)).alias(sensor)
    elif method == "max":
        agg_expr = spark_max(col(sensor)).alias(sensor)
    else:  # mean
        agg_expr = spark_avg(col(sensor)).alias(sensor)

    result = (
        joined.groupBy("target_when")
              .agg(agg_expr)
    )

    # 4. Final formatting
    final = (
        result
          .withColumnRenamed("target_when", "when")
          .withColumn("time", from_unixtime(col("when")))
          .orderBy("when")
    )

    return final

In [ ]:
# Nearest‐time fill
df_near = resample_sensor(df_hard, "S117", interval=60, method="near")
df_near.show(5)

# Minimum‐value fill
df_min = resample_sensor(df_hard, "S117", interval=60, method="min")
df_min.show(5)

# Maximum‐value fill
df_max = resample_sensor(df_hard, "S117", interval=60, method="max")
df_max.show(5)

# Mean‐value fill
df_mean = resample_sensor(df_hard, "S117", interval=60, method="mean")
df_mean.show(5)

# Invalid method raises:
# resample_sensor(df_hard, "S117", 60, method="median")
# → ValueError: Unsupported method 'median'. Choose from 'near', 'min', 'max', 'mean'.

In [ ]:
# PROVA MARCO VECCHIA CONVERSIONE SENSORI A5 A9, ricopiata il codice può essere utile
# --- OCCHIO --- questo ti pare che faccia la conversione del binario da dx verso sx, dovrebbe essere il contrario

# def convert_a5_to_binary_bits(df,sensor):
#     df_with_bits = df
    
#     for i in range(16):
#         bit_position = 15 - i  # S1 = bit 15 (leftmost), S16 = bit 0 (rightmost)
#         df_with_bits = df_with_bits.withColumn(
#             f"YOLO{i+1}",
#             (  col(sensor).bitwiseAND(1 << bit_position)  > 0 ).cast("int")
#         )
    
#     return df_with_bits

# # Usage
# df_with_binary = convert_a5_to_binary_bits(df_pivoted,"A5")

### Anomaly detection (Prova Marco sarà da cancellare)

In [ ]:
def detect_switch_anomaly(df, sensor, window_minutes=10, switch_threshold=5):
    """
    Detects frequent switching for binary sensor, partitioned by day to avoid single-node pressure.
    """

    # Extract day (or hour) from timestamp to partition
    df = df.withColumn("day", to_date(from_unixtime(col("when"))))

    # Lag to get previous value within each partition --- OTHERWISE IT MOVES ALL IN THE SAME PARTITION
    w_lag = Window.partitionBy("day").orderBy("when")
    df_lagged = df.withColumn("prev", lag(col(sensor)).over(w_lag))

    # Detect change 0→1 or 1→0
    df_changes = df_lagged.withColumn(
        "change", when((col("prev") != col(sensor)), 1).otherwise(0)
    )

    # Rolling window over time with partition
    w_time = Window.partitionBy("day").orderBy("when").rangeBetween(-window_minutes * 60, 0)
    df_windowed = df_changes.withColumn(
        "change_count", sum_("change").over(w_time)
    )

    # Flag anomaly
    df_anomaly = df_windowed.withColumn(
        "is_anomaly", (col("change_count") > switch_threshold)
    ).drop("prev", "change", "change_count", "day")

    return df_anomaly

In [ ]:
df_anom = detect_switch_anomaly(
  df_resampled_S117,
  sensor="S117",
  window_minutes=60,
  switch_threshold=6
)
df_anom.filter("is_anomaly").show()

In [ ]:
ini = 1602882742-50*60

In [ ]:
fine =1602882742+50*60

In [ ]:
inspect(df_resampled_S117, sensors=["S117"], start=ini, end=fine).show(100)

# Predictive Maintenance

# *** Remember to close Spark Session ***

In [ ]:
spark.stop()